# 大差がつく相手の分布

In [6]:
import pandas as pd
import numpy as np
import re
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from itertools import product
from scipy import stats

%matplotlib inline
plt.style.use('ggplot')
#plt.style.use('seaborn-pastel')
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tq

import matplotlib as mpl
font = {"family":"osaka"}
mpl.rc('font', **font)

sys.path.append("..")
from utils.util_functions import *

pd.set_option('display.max_rows', 500)

In [7]:
#event_df = make_df(data_dir=DETAIL_DATA_DIR_MLB)
#event_df = pd.read_csv(os.path.join(ROOT_DIR, "event_df.csv"), index_col=0, encoding="cp932", dtype=object)
#df_2017 = make_df(end=1944417, data_dir=DETAIL_DATA_DIR_MLB)
#df_2017 = pd.read_csv(os.path.join(ROOT_DIR, "df_2017.csv"), index_col=0, encoding="cp932", dtype=object)
#df_2018 = make_df(start=1944417, data_dir=DETAIL_DATA_DIR_MLB)
#df_2018 = pd.read_csv(os.path.join(ROOT_DIR, "df_2018.csv"), index_col=0, encoding="cp932", dtype=object)
files = os.listdir(DETAIL_DATA_DIR_MLB)[1:]
names = pd.Series(files).apply(lambda x: int(x.split(".")[0]))
files_2017 = names[names<=1944417].apply(lambda x: str(x)+".csv")
files_2018 = names[names>1944417].apply(lambda x: str(x)+".csv")
column = "得点"
teams_mlb = list(set(pd.read_csv(os.path.join(SCORE_DATA_DIR_MLB, "score_mlb.csv"), encoding="cp932", index_col=0).表チーム))
teams_mlb.remove("ア・リーグ")
first_teams = ['Rソックス', 'インディアンス', 'アストロズ', 'ブレーブス', 'ブリュワーズ', 'ドジャース']
second_teams = ['ヤンキース', 'ツインズ', 'アスレチックス', 'ナショナルズ', 'カブス', 'ロッキーズ']
third_teams = ['レイズ', 'タイガース', 'マリナーズ', 'フィリーズ', 'カージナルス', 'Dバックス']
fourth_teams = ['ブルージェイズ', 'Wソックス',  'エンゼルス', 'メッツ', 'パイレーツ', 'ジャイアンツ']
fifth_teams = ['オリオールズ', 'ロイヤルズ', 'レンジャーズ', 'マーリンズ', 'レッズ', 'パドレス']

In [8]:
score_df = pd.read_csv(os.path.join(SCORE_DATA_DIR_MLB, "score_mlb.csv"), encoding="cp932", index_col=0)

In [9]:
def func(team, n=5):
    top_leads = np.array(score_df[(score_df.表チーム==team)]["表得点"] - score_df[(score_df.表チーム==team)]["裏得点"])
    top_opps = np.array(score_df[(score_df.表チーム==team)]["裏チーム"])
    bot_leads = np.array(score_df[(score_df.裏チーム==team)]["裏得点"] - score_df[(score_df.裏チーム==team)]["表得点"])
    bot_opps = np.array(score_df[(score_df.裏チーム==team)]["表チーム"])
    leads = np.concatenate([top_leads, bot_leads])
    opps = np.concatenate([top_opps, bot_opps])
    df = pd.DataFrame([leads, opps], index=["リード", "相手チーム"]).T
    return df[df.リード>=n]

In [10]:
# 大差をつける相手として、有意に分布が違うチームはなさそう
# プロだからある程度接戦になると説明するべきか
for team in teams_mlb:
    opps = score_df[(score_df.表チーム==team) | (score_df.裏チーム==team)][["表チーム", "裏チーム"]].values.flatten()
    opps = opps[opps!=team]
    lead_opps = np.array(func(team).相手チーム)
    print(team, stats.ks_2samp(opps, lead_opps).pvalue)

オリオールズ 0.125021564496
レンジャーズ 0.971326954903
インディアンス 0.862600158327
マーリンズ 0.995804417892
ブルージェイズ 0.469885403521
パイレーツ 0.830513905956
ロイヤルズ 0.946974592358
ドジャース 0.0671422037667
レイズ 0.34983094668
ロッキーズ 0.994925038863
フィリーズ 0.353858647478
エンゼルス 0.948024660279
マリナーズ 0.342838945366
Wソックス 0.413546728639
タイガース 0.493048337129
パドレス 0.281395653032
カージナルス 0.169417258144
カブス 0.737445272124
ブレーブス 0.504889391146
レッズ 0.414123522423
ブリュワーズ 0.797652966023
ナショナルズ 0.112207412744
Rソックス 0.99608320391
アスレチックス 0.991539385442
ヤンキース 0.325647485252
メッツ 0.790584548442
アストロズ 0.99899319312
ツインズ 0.414559032894
Dバックス 0.134287673458
ジャイアンツ 0.674760503036


# positive vs negative

In [43]:
idx = 2

In [44]:
lst = make_flattened_list(make_inning_triple(make_inning_list(event_df[event_df.攻撃チーム==teams_mlb[idx]], score_end=1, how="own")))

In [45]:
improved_list = [elem for elem in lst if is_improved(elem[0][0], elem[1][0])]
deteriorated_list = [elem for elem in lst if is_deteriorated(elem[0][0], elem[1][0])]

In [56]:
decode_count("0000")

'0アウトランナーなし'

In [60]:
pd.DataFrame(sorted([decode_count(case) for case in set([elem[1][0] for elem in deteriorated_list])]))

,0
0,1アウト12塁
1,1アウト13塁
2,1アウト1塁
3,1アウト23塁
4,1アウト2塁
5,1アウト3塁
6,1アウトランナーなし
7,1アウト満塁
8,2アウト12塁
9,2アウト13塁


In [46]:
common_case_list = list(set([elem[1][0] for elem in improved_list]) & set([elem[1][0] for elem in deteriorated_list]))

In [47]:
case = common_case_list[0]
#[elem[2][1]-elem[1][1] for elem in improved_list if elem[1][0]==case]

In [48]:
imp_proba = (pd.Series([elem[1][0] for elem in improved_list]).value_counts()/len(pd.Series([elem[1][0] for elem in improved_list])))[common_case_list]
det_proba = (pd.Series([elem[1][0] for elem in deteriorated_list]).value_counts()/len(pd.Series([elem[1][0] for elem in deteriorated_list])))[common_case_list]

In [49]:
#det_proba

In [50]:
res_dict = dict()
for case in common_case_list:
    imp_score = np.mean([elem[2][1]-elem[1][1] for elem in improved_list if elem[1][0]==case])
    det_score = np.mean([elem[2][1]-elem[1][1] for elem in deteriorated_list if elem[1][0]==case])
    res_dict[case] = (imp_score, det_score)

In [51]:
pd.DataFrame(list(res_dict.values()), index=res_dict.keys(), columns=["imp", "det"])#.describe()

,imp,det
2100,0.068783,0.100865
1100,0.097107,0.064407
2110,0.224390,0.269504
1010,0.161677,0.181818
2011,0.266667,0.428571
1110,0.200855,0.311688
2101,0.239583,0.378378
2000,0.027027,0.033763
1101,0.578313,0.636364
2010,0.166667,0.217877


In [52]:
# positive = is_improved
sum(imp_proba * pd.DataFrame(list(res_dict.values()), index=res_dict.keys(), columns=["imp", "det"])["imp"])

0.13001310615989514

In [53]:
# negative = is_deteriorated
sum(det_proba * pd.DataFrame(list(res_dict.values()), index=res_dict.keys(), columns=["imp", "det"])["det"])

0.08217054263565891